In [2]:
from google.colab import drive
drive.mount('/content/drive/')      

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectPercentile, f_classif
import pandas as pd
from sklearn import metrics
from sklearn.feature_selection import chi2
import nltk
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import numpy as np
import time
import re

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
root_path = '/content/drive/My Drive/NB_Data/'
df_train_data = pd.read_csv(root_path+"training.1600000.processed.noemoticon.csv",encoding='latin-1', usecols=[0,5], header=None)
df_test_data  = pd.read_csv(root_path+"testdata.manual.2009.06.14.csv",encoding='latin-1', usecols=[0,5], header=None)

In [0]:
def clean_data(tweet):
  stopwords = nltk.corpus.stopwords.words('english')
  lemma = WordNetLemmatizer()
  tokens_list = re.split('\W+',tweet) 
  lower_list = [word.lower() for word in tokens_list if word.isalpha()]   
  txt_clean = " ".join([lemma.lemmatize(x) for x in lower_list if x not in stopwords])
  return txt_clean


In [7]:
#cleaning the training data
t1 = time.time()
df_train_data['clean_data'] = df_train_data[5].apply(lambda x : clean_data(x))
print('\n\nTime to clean training data = ',time.time()-t1)



Time to clean training data =  286.91118240356445


In [8]:
#cleaning the testing data
t1 = time.time()
df_test_data['clean_data'] = df_test_data[5].apply(lambda x : clean_data(x))
print('\n\nTime to clean testing data = ',time.time()-t1)



Time to clean testing data =  0.10615253448486328


In [0]:
#Converting cleand data into list for faster processing
features_train = df_train_data['clean_data'].tolist()
labels_train = df_train_data[0].tolist()

df_test_data = df_test_data[df_test_data[0]!=2]
features_test = df_test_data['clean_data']
labels_test = df_test_data[0]

In [0]:
vectorizer = TfidfVectorizer(sublinear_tf=True, dtype=np.float32)
X_train = vectorizer.fit_transform(features_train)
X_test = vectorizer.transform(features_test)

In [0]:
def data_slice(X,Y,i,batch_size):
    return X[i:i+batch_size], Y[i:i+batch_size]

In [0]:
def train_model(train_data_parm, model_name):
    batch_size = 1000
    for i in tqdm(range(1600)):
        X_slice,Y_slice  = data_slice(train_data_parm,labels_train,i*batch_size,batch_size)
        model_name.partial_fit(X_slice.todense(),Y_slice,classes=[0,4])
    

In [0]:
def cal_accuracy(model_name,test_data):
    score_test = model_name.score(test_data.todense(), labels_test)
    print('shape of test data :',test_data.shape)
    print("Testing data score:", score_test)

In [0]:
model1 = GaussianNB()

In [15]:
t0 = time.time()
train_model(X_train,model1)
print('Time to train the model without features selection = ',time.time()-t0)

100%|██████████| 1600/1600 [1:50:43<00:00,  4.16s/it]

Time to train the model without features selection =  6643.0398206710815


In [16]:
#training the model on the data without features selection
t0 = time.time()
cal_accuracy(model1,X_test)
print('Prediction time on test without features selection = ',time.time()-t0)

shape of test data : (359, 520119)
Testing data score: 0.48746518105849584
Prediction time on test without features selection =  2.5561695098876953


In [0]:
sp_model = SelectPercentile(chi2, percentile=10)
X_new_train = sp_model.fit_transform(X_train,labels_train)
X_new_test = sp_model.transform(X_test)

In [0]:
#creating new model
model2 = GaussianNB()

In [19]:
#training the model on the data without features selection
t0 = time.time()
train_model(X_new_train,model2)
print('\n\nTime to train the model with features selection = ',time.time()-t0)

100%|██████████| 1600/1600 [12:12<00:00,  2.20it/s]



Time to train the model with features selection =  732.8885593414307


In [20]:
#testing accuracy of the model on the data with features selection
t0 = time.time()
cal_accuracy(model2,X_new_test)
print('\n\nTime to check accuracy on testing data with features selection = ',time.time()-t0)

shape of test data : (359, 52012)
Testing data score: 0.5013927576601671


Time to check accuracy on testing data with features selection =  0.3007667064666748
